In [155]:
import pandas as pd
import json

In [71]:
"""

Travel Chat Bot for Slack
title:  amadeus_query.py
author: George Thoraninh
usage:  python amadeus_query.py

"""
# import statements
import os
import ssl
import requests
from functools import lru_cache

amadeus_api_key = os.environ['AMADEUS_API_KEY']
amadeus_api_secret = os.environ['AMADEUS_API_SECRET']


auth_response = requests.post('https://api.amadeus.com/v1/security/oauth2/token',
                              data={'grant_type': 'client_credentials',
                                    'client_id': amadeus_api_key,
                                    'client_secret': amadeus_api_secret})
bearer = auth_response.json()['access_token']
#print(bearer)
# API has quotas, so better to cache everything
@lru_cache(maxsize=2048)
def call_api(url, **params):
    full_url = f'https://api.amadeus.com/v1{url}'
    response = requests.get(full_url,
                            params=params,
                            headers={'Authorization': f'Bearer {bearer}',
                                     'Content-Type': 'application/vnd.amadeus+json'})
    return response.json()

response = call_api('/shopping/flight-offers',
                origin='YYZ',
                destination='NYC',
                departureDate='2020-08-01',
                returnDate='2020-08-15',
                adults='1',
                currency='CAD',
                maxPrice='1000',
                max='1'
                )
print(response.keys())

dict_keys(['data', 'dictionaries', 'meta'])


In [119]:
df = pd.DataFrame(response['data'])

In [154]:
response

{'data': [{'type': 'flight-offer',
   'id': '1586763857613--519338455',
   'offerItems': [{'services': [{'segments': [{'flightSegment': {'departure': {'iataCode': 'YYZ',
           'terminal': '3',
           'at': '2020-08-01T09:35:00-04:00'},
          'arrival': {'iataCode': 'LGA',
           'terminal': 'C',
           'at': '2020-08-01T11:04:00-04:00'},
          'carrierCode': 'WS',
          'number': '1202',
          'aircraft': {'code': '73H'},
          'operating': {'carrierCode': 'WS', 'number': '1202'},
          'duration': '0DT1H29M'},
         'pricingDetailPerAdult': {'travelClass': 'ECONOMY',
          'fareClass': 'E',
          'availability': 7,
          'fareBasis': 'LT7D0ZBG'}}]},
      {'segments': [{'flightSegment': {'departure': {'iataCode': 'LGA',
           'terminal': 'C',
           'at': '2020-08-15T06:05:00-04:00'},
          'arrival': {'iataCode': 'YYZ',
           'terminal': '3',
           'at': '2020-08-15T07:36:00-04:00'},
          'carrierCode

In [120]:
df

,type,id,offerItems
0,flight-offer,1586763857613--519338455,[{'services': [{'segments': [{'flightSegment':...


In [143]:
test = str(df['offerItems'][0])[1:-1] 

In [148]:
test2 = json.loads(test.replace('\'','\"'))

In [156]:
test2['price']['total']

'166.92'